In [4]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [5]:
from torchvision import models
from torchvision import transforms
import os

In [6]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning" or os.path.split(os.getcwd())[1] == "semi-supervised" or os.path.split(os.getcwd())[1] == "self-supervised"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

Current Working Directory: progettoVIPM


In [7]:
cuda=True

In [8]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split



trainM = ImageDataset(dataset=datasets.TRAINING_MIXED,network_input_size=256,cuda = cuda)
trainM = DataLoader(trainM,128,shuffle=False)

In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # (B, 64, H/2, W/2)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (B, 64, H/4, W/4)
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), # (B, 128, H/8, W/8)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (B, 128, H/16, W/16)
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1), # (B, 256, H/32, W/32)
            nn.ReLU()
        )
    def forward(self, x):
        return self.encoder(x)


class ExtendedEncoder(nn.Module):
    def __init__(self, base_encoder):
        super(ExtendedEncoder, self).__init__()
        self.base_encoder = base_encoder

        # Dynamically compute flatten_dim
        dummy_input = torch.randn(1, 3, 256, 256).to(device)  # Simulate input
        dummy_output = self.base_encoder(dummy_input)  # Get encoder output
        self.flatten_dim = dummy_output.view(1, -1).size(1)  # Flatten dimension
        
        # Fully connected layer
        self.fc = nn.Linear(self.flatten_dim, 251)
        self.softmax = nn.Softmax(dim=1)
        
        for param in self.base_encoder.parameters():
            param.requires_grad = False


    def forward(self, x):
        x = self.base_encoder(x)  # Base encoder
        x = x.view(x.size(0),-1)  # Flatte
        x = self.fc(x)  # Fully connected layer
        x = self.softmax(x)
        return x

In [10]:
PATH = "self-supervised\models\AutoEncoderModelV2.pth"

model = torch.load(PATH)

In [11]:
encoder = model.base_encoder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder.to(device)

Encoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
  )
)

In [14]:
for m in trainM:
    print(m)

AttributeError: 'NoneType' object has no attribute 'astype'

In [12]:
feature_train = []
encoder.eval()
with torch.no_grad():
    for inputs,_ in trainM:
        inputs = inputs.to(device)
        outputs = encoder(inputs)
        outputs = outputs.view(outputs.size(0),-1)  # Flatte
        feature_train.extend(outputs.cpu().numpy())
        

AttributeError: 'NoneType' object has no attribute 'astype'